# PyTorch Mini Lab

In [23]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### Network Definition

In [24]:
##### This network design is too shallow for the task, your job is to make it deeper! #####

class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5, padding=2)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv4 = nn.Conv2d(256, 256, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.pool4 = nn.MaxPool2d(2, 2)
        self.conv6 = nn.Conv2d(512, 512, 3, padding=1)
        self.pool5 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(512 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 37)
        self.ReLU = nn.ReLU()

    def forward(self, x):
        x = self.ReLU(self.conv1(x))
        x = self.pool1(x)
        x = self.ReLU(self.conv2(x))
        x = self.pool2(x)
        x = self.ReLU(self.conv3(x))
        x = self.ReLU(self.conv4(x))
        x = self.pool3(x)
        x = self.ReLU(self.conv5(x))
        x = self.pool4(x)
        x = self.ReLU(self.conv6(x))
        x = self.pool5(x)
        x = torch.flatten(x, start_dim=1)
        x = self.ReLU(self.fc1(x))
        x = self.fc2(x)
        return x

model = MyNetwork().to(device)

### Dataset and Training Loop

In [25]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.RandomCrop((256,256),pad_if_needed=True),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.OxfordIIITPet("pets_data/","trainval",transform=transform,download=True)
test_dataset = torchvision.datasets.OxfordIIITPet("pets_data/","test",transform=transform,download=True)

train_loader = DataLoader(train_dataset,batch_size=8,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=8)


learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_func = torch.nn.CrossEntropyLoss()

num_epochs = 5

for epoch in range(num_epochs):
    print("Epoch:", epoch)
    for im, label in train_loader:
        # Move input data and labels to GPU if available
        im, label = im.to(device), label.to(device)

        optimizer.zero_grad()
        output = model(im)
        loss = loss_func(output, label)
        print(loss)
        loss.backward()
        optimizer.step()


Epoch: 0
tensor(3.6234, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6090, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6054, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6192, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.5755, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6194, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6665, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6141, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.5800, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6063, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6209, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6495, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6185, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6192, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6079, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.5986, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.5934, device='cuda:0',

### Testing

In [27]:
correct = 0
total = 0

model.eval()
with torch.no_grad():
    for im, label in test_loader:
        # Move input data and labels to GPU if available
        im, label = im.to(device), label.to(device)

        output = model(im)
        guess = torch.argmax(output, dim=1)
        correct += torch.sum(guess == label).item()
        total += len(guess)

print("Testing Accuracy is", correct / total * 100, "%")


Testing Accuracy is 14.52711910602344 %
